In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
df = pd.read_csv('../Data/heart_disease_uci.csv')

In [4]:
x_regression = df.drop(columns=['chol', 'num'])
y_regression = df['chol']


In [6]:
x_train_reg, x_test_reg, y_train_reg, y_test_reg = train_test_split(x_regression, y_regression, test_size=0.2, random_state=42)

In [ ]:
# Check if `num` is binary or multiclass
print(df['num'].value_counts())

# Optionally convert to binary classification (0 = no disease, 1 = disease)
# You can skip this if you want multiclass classification
df['num'] = df['num'].apply(lambda x: 1 if x > 0 else 0)

# Features and target
X_clf = df.drop(columns=['num', 'chol'])  # remove targets
y_clf = df['num']

# Split
X_train_clf, X_test_clf, y_train_clf, y_test_clf = train_test_split(
    X_clf, y_clf, test_size=0.2, random_state=42
)
